In [ ]:
!pip install -q pyngrok
!pip install -q streamlit
!pip install -q streamlit_ace
!pip install SpeechRecognition
!pip install PyAudio

In [ ]:
!git clone https://github.com/stefanrmmr/streamlit_audio_recorder.git

In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
%%writefile app.py

import re
import os
import streamlit as st
from pyngrok import ngrok
import numpy as np
import datetime
import streamlit.components.v1 as components
from io import BytesIO
import speech_recognition as sr
import soundfile as sf
import librosa

parent_dir = os.path.dirname(os.path.abspath(__file__))
build_dir = os.path.join(parent_dir, "streamlit_audio_recorder/st_audiorec/frontend/build")
st_audiorec = components.declare_component("st_audiorec", path=build_dir)

#Essa função é para criação da webpage
def main():

  #Elementos da webpage
  html_temp = """
  <div>
    <div style ="width: 34%; margin-right: 1%; height: 100px; display: inline-block; float: left; background-image: url(https://upload.wikimedia.org/wikipedia/commons/thumb/7/77/Logo_C6_Bank.svg/2560px-Logo_C6_Bank.svg.png); background-repeat: no-repeat; background-position-x: center; background-position-y: center; background-size: 90%;">
    </div>
    <div style ="background: #000; width: 65%; height: 100px; display: inline-block; padding: 23px;">
      <h3 style ="color: white; text-align: center;">Área PIX<h3>
    </div>
  </div>
  """

  def foraHorario(hora):
    hora = int(hora)

    if hora == 0:
      hora = 21
    elif hora == 1:
      hora = 22
    elif hora == 2:
      hora = 23
    elif hora == 3:
      hora = 0
    else:
      hora = hora - 3

    if 20 <= hora or hora < 6:
      return True
    else:
      return False

  def validarAudio():
    audio_source = '/content/audio.wav'
    data, sample_rate = librosa.load(audio_source, sr = None)
    sf.write(audio_source, data, sample_rate)
    audio = sr.AudioFile(audio_source)
    with audio as source:
      sr.Recognizer().adjust_for_ambient_noise(source, duration=0.1)
      texto = sr.Recognizer().record(source)

    retorno = False
    try:
      frase = sr.Recognizer().recognize_google(texto, language='pt-BR')
      if frase == 'eu fui à praia ontem':
        retorno = True
    except sr.UnknownValueError:
      retorno = 'err'

    return retorno

    if sr.Recognizer().recognize_google(texto, language='pt-BR') == 'eu fui à praia ontem':
      return True
    else:
      return false

  # Função do stramlit que faz o display da webpage
  st.markdown(html_temp, unsafe_allow_html = True) 

  # As linhas abaixo criam as caixas na qual o usuário vai entrar com dados da pessoa que quer o empréstimo para fazer a Predição
  col1, col2 = st.columns(2)
  with col1:
    chave = st.text_input('Chave PIX*')

  with col2:
    valor = st.text_input('Valor*')

  st.text('Envie um audio com a frase "Eu fui à praia ontem".')
  audio = st_audiorec()

  if isinstance(audio, dict):  # retrieve audio data
    with st.spinner('retrieving audio-recording...'):
      ind, audio = zip(*audio['arr'].items())
      ind = np.array(ind, dtype=int)  # convert to np array
      audio = np.array(audio)             # convert to np array
      sorted_ints = audio[ind]
      stream = BytesIO(b"".join([int(v).to_bytes(1, "big") for v in sorted_ints]))
      wav_bytes = stream.read()

      with open('audio.wav', 'wb') as f:
        f.write(wav_bytes)

  def validation(chave, valor):
    if st.button("Transferir"):
      if valor.isnumeric() and int(valor) > 0:
        if foraHorario(datetime.datetime.now().strftime("%H")) and int(valor) > 3000:
          if validarAudio():
            st.success('Transação autorizada')  
          else:
            st.error('Transação não autorizada')
        else:
          st.success('Transação autorizada')
      else:
        st.warning('Preencha um valor válido')

  validation(chave, valor)

if __name__=='__main__': 
  main()

Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501